In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tictactoe
from tictactoe import calc_reward_numba, MASK_L

import numpy as np

## 벨만 기대 방적식을 통해 Qsa 구하기

In [3]:
def fact(N):
    return np.prod(range(1,N+1))

def comb(N: int, M: int):
    if M == 0:
        return 0
    else:
        return np.prod(range(N-M+1, N+1))

In [4]:
comb(9,2)

72

In [5]:
comb(9,9)

362880

In [22]:
class ActionValueFunc:
    def __init__(self):
        self.Qsa = [
            [], # O 플레이어
            []  # X 플레이어 
        ]

        # O 플레이어를 위한 Qsa 함수값 초기화 
        for stage in range(5):
            n_positions = comb(9,stage*2)
            self.Qsa[0].append(np.zeros(n_positions))
            print(self.Qsa[0][stage].shape)
            
        # O 플레이어를 위한 Qsa 함수값 초기화 
        for stage in range(4):
            n_positions = comb(9,1 + stage*2)
            self.Qsa[1].append(np.zeros(n_positions))
            print(self.Qsa[1][stage].shape)
            
    def update_Qsa(self, player, stage, position_list, action):
        """
        먼저 시작한 O 플레이어의 Qsa[S,A]를 구하기 위해서는 Action에 따라 
        새로운 O가 놓여지는 위치에 따라 게임이 승리로 끝났는지 살핀다
        게임이 끝나지 않았다면 action까지 포함한 전체 돌의 수가 9 보다 적은지 살핀다. 
        상기 상황이 맞다면 상대방 돌인 X의 위치를 나머지에 포진 시킨다. 
        이때 상대방은 일단 무작위 정책을 사용한다 가정한다.
        무작위 정책에 더불어 반드시 놓을 곳, 막아야 하는 곳은 인지한다는 가정하의 정책도 고려할 예정임.
        이렇게 필수 규칙은 O 플레이어의 경우에도 적용해 Q[S,A]를 구하게 하는 것도 가능할거 같음. (검증 필요)
        예를 들어 첫 스테이지라면 O는 9가지중에 1가지로 정해졌지만, X의 위치에 따른 8가지 상태는 고려되어야 함. 
        X의 위치별로 reward가 정해지게 되고 (이때 X위치는 무작위로 확률값을 우선 계산하지 않았음),
        X의 위치 8개 각각에 대해 sum_A PI(A|S)*Q[S,A] for k+1를 구해 Qsa[S,A] 값의 업데이에 반영함. 
        """
        assert len(position_list) < 9, 'len(position_list) should be less than 9'
        def calc_reward(position_list, action):
            S = np.zeros(9, dtype=int)
            now_player = player + 1
            for p in position_list:
                S[p] = now_player
                now_player = 1 if now_player == 2 else 2
                #print('S =', S)
            # assert now_player == player + 1
            S[action] = now_player 
            print('S = ', S)
            reward = calc_reward_numba(MASK_L, S)
            #print(reward)
            reward = -1 if reward == 2 else reward
            #print(reward)
            return reward

        if player == 0:
            reward = calc_reward(position_list, action) 
            # Qsa_k1 is not exist  
            if reward == 0 and len(position_list) + 1 < 9:
                op_action_list = action_list.copy()
                op_action_list.remove(action)
                reward_list = []
                for op_action in op_action_list:
                    # 만약, 상대방의 정책이 무작위가 아니라 non-uniform이라면 해당 값을 곱해야 함.
                    reward_i = calc_reward(position_list + [action], op_action) # * op_policy(state, op_action)
                    reward_list.append(reward_i)
                reward = np.mean(reward_list)
            print(reward)  
        
# def get_Qsa(Qsa, player, stage, position, action):  

Qsa = ActionValueFunc()

(0,)
(72,)
(3024,)
(60480,)
(362880,)
(9,)
(504,)
(15120,)
(181440,)


In [23]:
comb(9,7)

181440

In [46]:
def get_Qsa(Qsa, player, stage, position_list, action_list):
    def calc_reward(position_list, action):
        S = np.zeros(9, dtype=int)
        now_player = player + 1
        for p in position_list:
            S[p] = now_player
            now_player = 1 if now_player == 2 else 2
            #print('S =', S)
        # assert now_player == player + 1
        S[action] = now_player 
        print('S = ', S)
        reward = calc_reward_numba(MASK_L, S)
        #print(reward)
        reward = -1 if reward == 2 else reward
        #print(reward)
        return reward
        
    if player == 0:
        # calc reward
        # Qsa[player, stage, position] = 
        for action in action_list:
            # E[r + gamma*Qsa]을 계산할 떄 각 policy(state, action)을 전체에 곱해야 함
            # 즉, (r + gamma*Qsa) * policy(state, action)이 되어야 함. 
            # policy(state, action)이 만약 0라면 계산에 반영할 필요가 없음.
            reward = calc_reward(position_list, action) 
            if reward ==0 and len(position_list) + 1 < 9:
                op_action_list = action_list.copy()
                op_action_list.remove(action)
                reward_list = []
                for op_action in op_action_list:
                    # 만약, 상대방의 정책이 무작위가 아니라 non-uniform이라면 해당 값을 곱해야 함.
                    reward_i = calc_reward(position_list + [action], op_action) # * op_policy(state, op_action)
                    reward_list.append(reward_i)
                reward = np.mean(reward_list)
            print(reward)

Qsa = ActionValueFunc()
get_Qsa(Qsa, 0, 0, [], list(range(9)))

(0,)
(72,)
(3024,)
(60480,)
(362880,)
(9,)
(504,)
(15120,)
(181440,)
S =  [1 0 0 0 0 0 0 0 0]
S =  [1 2 0 0 0 0 0 0 0]
S =  [1 0 2 0 0 0 0 0 0]
S =  [1 0 0 2 0 0 0 0 0]
S =  [1 0 0 0 2 0 0 0 0]
S =  [1 0 0 0 0 2 0 0 0]
S =  [1 0 0 0 0 0 2 0 0]
S =  [1 0 0 0 0 0 0 2 0]
S =  [1 0 0 0 0 0 0 0 2]
0.0
S =  [0 1 0 0 0 0 0 0 0]
S =  [2 1 0 0 0 0 0 0 0]
S =  [0 1 2 0 0 0 0 0 0]
S =  [0 1 0 2 0 0 0 0 0]
S =  [0 1 0 0 2 0 0 0 0]
S =  [0 1 0 0 0 2 0 0 0]
S =  [0 1 0 0 0 0 2 0 0]
S =  [0 1 0 0 0 0 0 2 0]
S =  [0 1 0 0 0 0 0 0 2]
0.0
S =  [0 0 1 0 0 0 0 0 0]
S =  [2 0 1 0 0 0 0 0 0]
S =  [0 2 1 0 0 0 0 0 0]
S =  [0 0 1 2 0 0 0 0 0]
S =  [0 0 1 0 2 0 0 0 0]
S =  [0 0 1 0 0 2 0 0 0]
S =  [0 0 1 0 0 0 2 0 0]
S =  [0 0 1 0 0 0 0 2 0]
S =  [0 0 1 0 0 0 0 0 2]
0.0
S =  [0 0 0 1 0 0 0 0 0]
S =  [2 0 0 1 0 0 0 0 0]
S =  [0 2 0 1 0 0 0 0 0]
S =  [0 0 2 1 0 0 0 0 0]
S =  [0 0 0 1 2 0 0 0 0]
S =  [0 0 0 1 0 2 0 0 0]
S =  [0 0 0 1 0 0 2 0 0]
S =  [0 0 0 1 0 0 0 2 0]
S =  [0 0 0 1 0 0 0 0 2]
0.0
S =  [0 0 0 0 1

In [35]:
action = [1,2,3]
action.remove(2)

In [37]:
action 

[1, 3]